In [1]:
import pandas as pd
import numpy as np
import acquire as a
import prepare as p
import model_functions as m

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier



import warnings
warnings.filterwarnings('ignore')


## Prep for modeling

In [2]:
#import data and prep for modeling
df = a.get_heart()
df = p.rename_cols(df)
df = p.replace_cat_values(df)
df = m.df_classification_ready(df, 'sex_female')
df.head()

,age,resting_bp,cholesterol,fasting_blood_sugar>120,max_heart_rate,exercise_induced_angina,oldpeak,num_major_blood_vessels,high_risk_of_mi,sex_male,...,chest_pain_type_typical angina,rest_ecg_ST-T wave abnormal,rest_ecg_left ventricular hypertrophy,rest_ecg_normal,st_slope_downsloping,st_slope_flat,st_slope_unsloping,defect_type_fixed_defect,defect_type_normal,defect_type_reversible
0,63,145,233,1,150,0,2.3,0,1,1,...,0,0,0,1,0,0,1,1,0,0
1,37,130,250,0,187,0,3.5,0,1,1,...,0,1,0,0,0,0,1,0,1,0
2,41,130,204,0,172,0,1.4,0,1,0,...,1,0,0,1,1,0,0,0,1,0
3,56,120,236,0,178,0,0.8,0,1,1,...,1,1,0,0,1,0,0,0,1,0
4,57,120,354,0,163,1,0.6,0,1,0,...,0,1,0,0,1,0,0,0,1,0


In [3]:
def df_classification_ready(df, cols= None):
    df = pd.get_dummies(df)
    df = df.drop(columns= [cols])

    return df

In [4]:
# df_classification_ready(df, 'sex_female')

In [5]:
#split into train val test
train, val, test = p.split_data(df, 'high_risk_of_mi')
train.shape, val.shape, test.shape

((201, 23), (51, 23), (49, 23))

In [6]:
#isolating target variable in each dataset
X_Train, y_Train, X_val, y_val, X_test, y_test = m.isolate_target(train, val, test, 'high_risk_of_mi')
X_Train.shape, y_Train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((201, 22), (201,), (51, 22), (51,), (49, 22), (49,))

Data is ready for modeling.

Got dummies, dropped duplicates, split to train, val, test, isolated target variable for train, val, test

# Perform modeling

#### Decision Tree

In [7]:
#get mode baseline 
y_Train.value_counts()


1    109
0     92
Name: high_risk_of_mi, dtype: int64

In [8]:
X_Train['baseline_prediction'] = 1


In [11]:
#function to run multiple random forest to compare for best accuracy
#Deprecated, function changed to calculate recall and updated in model_functions.py
'''
def get_decision_tree_multiple(X_Train, y_Train, X_val, y_val):
    metrics = []

    for j in range (1, 10):
        for i in range(2, 10):
            clf = DecisionTreeClassifier(max_depth=i, min_samples_leaf=j, random_state=123)

            clf = clf.fit(X_Train, y_Train)
            in_sample_accuracy = clf.score(X_Train, y_Train)
            out_of_sample_accuracy = clf.score(X_val, y_val)

            output = {
                "min_samples_per_leaf": j,
                "max_depth": i,
                "train_accuracy": in_sample_accuracy,
                "validate_accuracy": out_of_sample_accuracy
            }
    
            metrics.append(output)

    df1 = pd.DataFrame(metrics)
    df1["difference"] = df1.train_accuracy - df1.validate_accuracy
    df1_sorted = df1.sort_values(by=['validate_accuracy'], ascending=False).head(10)

    return df1_sorted
'''


'\ndef get_decision_tree_multiple(X_Train, y_Train, X_val, y_val):\n    metrics = []\n\n    for j in range (1, 10):\n        for i in range(2, 10):\n            clf = DecisionTreeClassifier(max_depth=i, min_samples_leaf=j, random_state=123)\n\n            clf = clf.fit(X_Train, y_Train)\n            in_sample_accuracy = clf.score(X_Train, y_Train)\n            out_of_sample_accuracy = clf.score(X_val, y_val)\n\n            output = {\n                "min_samples_per_leaf": j,\n                "max_depth": i,\n                "train_accuracy": in_sample_accuracy,\n                "validate_accuracy": out_of_sample_accuracy\n            }\n    \n            metrics.append(output)\n\n    df1 = pd.DataFrame(metrics)\n    df1["difference"] = df1.train_accuracy - df1.validate_accuracy\n    df1_sorted = df1.sort_values(by=[\'validate_accuracy\'], ascending=False).head(10)\n\n    return df1_sorted\n'

#plot results of decision tree results

x = 
y = 
hue = validate_accuracy

In [ ]:
best_decision_tree = pd.DataFrame((dec_tree_results.loc[27]))
best_decision_tree

#### Random Forest

In [ ]:
#function to run multiple random forest to compare for best accuracy
def get_random_forest_multiple(X_Train, y_Train, X_val, y_val):
    metrics = []

    for j in range (1, 10):
        for i in range(2, 10):
            rf = RandomForestClassifier(max_depth=i, min_samples_leaf=j, random_state=123)

            rf = rf.fit(X_Train, y_Train)
            in_sample_accuracy = rf.score(X_Train, y_Train)
            out_of_sample_accuracy = rf.score(X_val, y_val)

            output = {
                "min_samples_per_leaf": j,
                "max_depth": i,
                "train_accuracy": in_sample_accuracy,
                "validate_accuracy": out_of_sample_accuracy
            }
    
            metrics.append(output)

    df1 = pd.DataFrame(metrics)
    df1["difference"] = df1.train_accuracy - df1.validate_accuracy
    df1_sorted = df1.sort_values(by=['validate_accuracy'], ascending=False).head(10)

    return df1_sorted



In [ ]:
#showing results of decision tree models
rand_forest_results = get_random_forest_multiple(X_Train, y_Train, X_val, y_val)
rand_forest_results

Best performing Random Forest Model:

3 min_samples_per_leaf

9 max_depth

In [ ]:
best_random_forest = pd.DataFrame((rand_forest_results.loc[23]))
best_random_forest

#### KNN

In [ ]:
#function run multiple KNN to compare for best accuracy
def get_knn(X_Train, y_Train, X_val, y_val):
    metrics = []

    for i in range(2, 10):
        knn = KNeighborsClassifier(n_neighbors=i, weights='uniform')
        knn = knn.fit(X_Train, y_Train)
        in_sample_accuracy = knn.score(X_Train, y_Train)
        out_of_sample_accuracy = knn.score(X_val, y_val)

        output = {
            "neighbors": i,
            "train_accuracy": in_sample_accuracy,
            "validate_accuracy": out_of_sample_accuracy
        }

        metrics.append(output)

    df1 = pd.DataFrame(metrics)
    df1["difference"] = df1.train_accuracy - df1.validate_accuracy
    df1_sorted = df1.sort_values(by=['validate_accuracy'], ascending=False).head(10)

    return df1_sorted



In [ ]:
knn_results = get_knn(X_Train, y_Train, X_val, y_val)
knn_results

Best performing KNN Model:

5 neighbors

In [ ]:
best_knn= pd.DataFrame((knn_results.loc[3]))
best_knn

# Modeling Summary

Best performing model overall model was Random Forest with Training and Validate accuracy both at 94%.